## VARIABLES

In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
basepath = "dades_tfg/data"
demographics = pd.read_csv(os.path.join(basepath, "demographics.csv"))
demographics_N = pd.read_csv(os.path.join(basepath, "demographics_N.csv"))
nodes = pd.read_csv(os.path.join(basepath, "nodes.csv"))
basepath_FA = os.path.join(basepath, "FA")
basepath_GM = os.path.join(basepath, "GM")
basepath_RS = os.path.join(basepath, "RS")

In [3]:
filenames = ["{:04d}.csv".format(x) for x in demographics["id"]]

In [4]:
data_FA = np.zeros(shape=(len(filenames), 76, 76))
data_GM = np.zeros(shape=(len(filenames), 76, 76))
data_RS = np.zeros(shape=(len(filenames), 76, 76))

In [5]:
for i, filename in enumerate(filenames):
    df_FA = pd.read_csv(os.path.join(basepath_FA, filename), header=None)
    data_FA[i,:,:] = df_FA.values
    
    df_GM = pd.read_csv(os.path.join(basepath_GM, filename), header=None)
    data_GM[i,:,:] = df_GM.values
    
    df_RS = pd.read_csv(os.path.join(basepath_RS, filename), header=None)
    data_RS[i,:,:] = df_RS.values

In [6]:
basepath_FA_N = "dades_tfg/data/Naples/DTI_networks"
basepath_GM_N = "dades_tfg/data/Naples/GM_networks"
basepath_RS_N = "dades_tfg/data/Naples/rsfmri_networks"

# Listar nombres de archivos CSV en cada carpeta
filenames_FA_N = sorted([f for f in os.listdir(basepath_FA_N) if f.endswith('.csv')])
filenames_GM_N = sorted([f for f in os.listdir(basepath_GM_N) if f.endswith('.csv')])
filenames_RS_N = sorted([f for f in os.listdir(basepath_RS_N) if f.endswith('.csv')])

In [7]:
# Inicializar arrays para almacenar los datos
data_FA_N = np.zeros((len(filenames_FA_N), 76, 76))  
data_GM_N = np.zeros((len(filenames_GM_N), 76, 76))
data_RS_N = np.zeros((len(filenames_RS_N), 76, 76))

In [8]:
# FA
for i, filename in enumerate(filenames_FA_N):
    df_FA_N = pd.read_csv(os.path.join(basepath_FA_N, filename), header=None)
    data_FA_N[i, :, :] = df_FA_N.values

#  GM
for i, filename in enumerate(filenames_GM_N):
    df_GM_N = pd.read_csv(os.path.join(basepath_GM_N, filename), header=None)
    data_GM_N[i, :, :] = df_GM_N.values

#  RS
for i, filename in enumerate(filenames_RS_N):
    df_RS_N = pd.read_csv(os.path.join(basepath_RS_N, filename), header=None)
    data_RS_N[i, :, :] = df_RS_N.values

In [9]:
num_samples_FA = len(data_FA)
num_samples_FA_N = len(data_FA_N)
num_samples_GM = len(data_FA)
num_samples_GM_N = len(data_FA_N)
num_samples_RS = len(data_FA)
num_samples_RS_N = len(data_FA_N)


data_FA_combined = np.zeros((num_samples_FA + num_samples_FA_N, 76, 76))  
data_FA_combined[:num_samples_FA, :, :] = data_FA
data_FA_combined[num_samples_FA:, :, :] = data_FA_N


data_GM_combined = np.zeros((num_samples_GM + num_samples_GM_N, 76, 76))  
data_GM_combined[:num_samples_GM, :, :] = data_GM
data_GM_combined[num_samples_GM:, :, :] = data_GM_N


data_RS_combined = np.zeros((num_samples_RS + num_samples_RS_N, 76, 76))  
data_RS_combined[:num_samples_RS, :, :] = data_RS
data_RS_combined[num_samples_RS:, :, :] = data_RS_N

In [10]:
demographics_N['mstype'] = demographics_N['mstype'].apply(lambda x: 0 if x == -1 else 1)

In [11]:
data_FA_combined.shape

(270, 76, 76)

## GNNS

In [12]:
import torch
import pandas as pd
import numpy as np
from torch_geometric.data import Data

# CAL DEFINIR ELS EMBEDDINGS ES FARAN SERVIR
# EN AQUEST CODI, NOMÉS ÉS UN VECTOR DE (node_dim X 1's)
node_dim = 1  # Asegúrate de que esto coincida con las dimensiones del archivo
#node_dim = 76 si fem els les metriques dels nodes

def array_to_graph(data, y, th=0.0):
    num_nodes = data.shape[0]
    
    edge_index = []
    edge_weight = []
    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                if data[i, j] >= th:
                    edge_index.append([i, j])
                    edge_weight.append(data[i, j])
        
    y = torch.tensor([int(y)], dtype=torch.long)
    edge_index = torch.tensor(edge_index, dtype=torch.long)
    
    # Cargar datos de embeddings desde el archivo CSV
    embeddings_df = pd.read_csv('csvs/average_clustering_fa.csv', header=None) #modificar csv a necessitat
    x = torch.tensor(embeddings_df.values, dtype=torch.float)

    edge_weight = torch.tensor(edge_weight, dtype=torch.float)
    
    data = Data(x=x, edge_index=edge_index.t().contiguous(), edge_weight=edge_weight, y=y)
    #print(x)
    return data


In [13]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_max_pool

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(node_dim, 64)
        self.lin1 = torch.nn.Linear(64, 2)

    def forward(self, data):
        x, edge_index, edge_weight, batch = data.x, data.edge_index, data.edge_weight, data.batch
        
        x = self.conv1(x, edge_index, edge_weight)
        x = F.relu(x)
        x = global_max_pool(x, batch)
        x = self.lin1(x)

        return x

device = torch.device('cpu')

In [14]:
data = data_FA_combined
print(data.shape)

target = demographics_N['mstype'].to_numpy()
print(target.shape)

(270, 76, 76)
(270,)


In [15]:
from torch_geometric.loader import DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve

skf = StratifiedKFold(n_splits=3)

NUM_EPOCHS = 20
preds = np.zeros(data.shape[0])
fold = 0

for train_index, test_index in skf.split(data, target):
    fold += 1
    print("Fold: {}".format(fold))

    # split dataset
    X_train, X_test = data[train_index], data[test_index]
    y_train, y_test = target[train_index], target[test_index]
    
    prop_train = np.where(y_train == 1)[0].shape[0] / y_train.shape[0]
    prop_test = np.where(y_test == 1)[0].shape[0] / y_test.shape[0]
    print("Train set size     : {}".format(X_train.shape))
    print("Test set size      : {}".format(X_test.shape))
    print("Train set % of pwMS: {:.4f} ({})".format(prop_train, y_train.sum()))
    print("Test set % of pwMS : {:.4f} ({})".format(prop_test, y_test.sum()))

    # list of Data structures (one for each subject)
    train_graphs = []
    for i in range(X_train.shape[0]):
        g = array_to_graph(X_train[i], y_train[i])
        train_graphs.append(g)
        
    test_graphs = []
    for i in range(X_test.shape[0]):
        g = array_to_graph(X_test[i], y_test[i])
        test_graphs.append(g)

    # create the model
    model = GCN()
    model = model.to(device)
    # optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    # loss function
    loss_fn = torch.nn.CrossEntropyLoss()

    # train function
    def train():
        model.train()
        
        train_loader = DataLoader(train_graphs, batch_size=32, shuffle=True)

        loss_all = 0
        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()
            output = model(batch)
            label = batch.y
            label = F.one_hot(label, num_classes=2)
            label = label.type(torch.FloatTensor)
            label = label.to(device)
            loss = loss_fn(output, label)
            loss.backward()
            loss_all += batch.num_graphs * loss.item()
            optimizer.step()

        return loss_all / len(train_graphs)

    # train for N epochs
    for epoch in range(NUM_EPOCHS):
        loss_value = train()
        print("Train loss at epoch {}: {:.4f}".format(epoch + 1, loss_value))

    # test phase 
    test_loader = DataLoader(test_graphs, batch_size=len(test_graphs), shuffle=False)
    
    for batch in test_loader:
        batch = batch.to(device)
        test_preds = F.softmax(model(batch), dim=1).detach().numpy()
    
    test_preds = test_preds[:, 1]
    preds[test_index] = test_preds
    
    auc_roc = roc_auc_score(y_test, test_preds)
    print("Test AUC: {:.2f}".format(auc_roc))


Fold: 1
Train set size     : (180, 76, 76)
Test set size      : (90, 76, 76)
Train set % of pwMS: 0.5444 (98)
Test set % of pwMS : 0.5444 (49)
Train loss at epoch 1: 0.6903
Train loss at epoch 2: 0.6896
Train loss at epoch 3: 0.6896
Train loss at epoch 4: 0.6900
Train loss at epoch 5: 0.6893
Train loss at epoch 6: 0.6894
Train loss at epoch 7: 0.6892
Train loss at epoch 8: 0.6893
Train loss at epoch 9: 0.6892
Train loss at epoch 10: 0.6892
Train loss at epoch 11: 0.6892
Train loss at epoch 12: 0.6893
Train loss at epoch 13: 0.6892
Train loss at epoch 14: 0.6894
Train loss at epoch 15: 0.6892
Train loss at epoch 16: 0.6894
Train loss at epoch 17: 0.6893
Train loss at epoch 18: 0.6895
Train loss at epoch 19: 0.6893
Train loss at epoch 20: 0.6892
Test AUC: 0.50
Fold: 2
Train set size     : (180, 76, 76)
Test set size      : (90, 76, 76)
Train set % of pwMS: 0.5444 (98)
Test set % of pwMS : 0.5444 (49)
Train loss at epoch 1: 0.6929
Train loss at epoch 2: 0.6920
Train loss at epoch 3: 0.691

In [16]:
import torch
import pandas as pd
import numpy as np
from torch_geometric.data import Data

# CAL DEFINIR ELS EMBEDDINGS ES FARAN SERVIR
# EN AQUEST CODI, NOMÉS ÉS UN VECTOR DE (node_dim X 1's)
node_dim = 1  # Asegúrate de que esto coincida con las dimensiones del archivo

def array_to_graph(data, y, th=0.0):
    num_nodes = data.shape[0]
    
    edge_index = []
    edge_weight = []
    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                if data[i, j] >= th:
                    edge_index.append([i, j])
                    edge_weight.append(data[i, j])
        
    y = torch.tensor([int(y)], dtype=torch.long)
    edge_index = torch.tensor(edge_index, dtype=torch.long)
    
    # Cargar datos de embeddings desde el archivo CSV
    embeddings_df = pd.read_csv('csvs/average_clustering_gm.csv', header=None) #modificar csv a necessitat
    x = torch.tensor(embeddings_df.values, dtype=torch.float)

    edge_weight = torch.tensor(edge_weight, dtype=torch.float)
    
    data = Data(x=x, edge_index=edge_index.t().contiguous(), edge_weight=edge_weight, y=y)
    #print(x)
    return data

In [17]:
data = data_GM_combined
print(data.shape)

target = demographics_N['mstype'].to_numpy()
print(target.shape)

(270, 76, 76)
(270,)


In [18]:
from torch_geometric.loader import DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve

skf = StratifiedKFold(n_splits=3)

NUM_EPOCHS = 20
preds = np.zeros(data.shape[0])
fold = 0

for train_index, test_index in skf.split(data, target):
    fold += 1
    print("Fold: {}".format(fold))

    # split dataset
    X_train, X_test = data[train_index], data[test_index]
    y_train, y_test = target[train_index], target[test_index]
    
    prop_train = np.where(y_train == 1)[0].shape[0] / y_train.shape[0]
    prop_test = np.where(y_test == 1)[0].shape[0] / y_test.shape[0]
    print("Train set size     : {}".format(X_train.shape))
    print("Test set size      : {}".format(X_test.shape))
    print("Train set % of pwMS: {:.4f} ({})".format(prop_train, y_train.sum()))
    print("Test set % of pwMS : {:.4f} ({})".format(prop_test, y_test.sum()))

    # list of Data structures (one for each subject)
    train_graphs = []
    for i in range(X_train.shape[0]):
        g = array_to_graph(X_train[i], y_train[i])
        train_graphs.append(g)
        
    test_graphs = []
    for i in range(X_test.shape[0]):
        g = array_to_graph(X_test[i], y_test[i])
        test_graphs.append(g)

    # create the model
    model = GCN()
    model = model.to(device)
    # optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    # loss function
    loss_fn = torch.nn.CrossEntropyLoss()

    # train function
    def train():
        model.train()
        
        train_loader = DataLoader(train_graphs, batch_size=32, shuffle=True)

        loss_all = 0
        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()
            output = model(batch)
            label = batch.y
            label = F.one_hot(label, num_classes=2)
            label = label.type(torch.FloatTensor)
            label = label.to(device)
            loss = loss_fn(output, label)
            loss.backward()
            loss_all += batch.num_graphs * loss.item()
            optimizer.step()

        return loss_all / len(train_graphs)

    # train for N epochs
    for epoch in range(NUM_EPOCHS):
        loss_value = train()
        print("Train loss at epoch {}: {:.4f}".format(epoch + 1, loss_value))

    # test phase 
    test_loader = DataLoader(test_graphs, batch_size=len(test_graphs), shuffle=False)
    
    for batch in test_loader:
        batch = batch.to(device)
        test_preds = F.softmax(model(batch), dim=1).detach().numpy()
    
    test_preds = test_preds[:, 1]
    preds[test_index] = test_preds
    
    auc_roc = roc_auc_score(y_test, test_preds)
    print("Test AUC: {:.2f}".format(auc_roc))


Fold: 1
Train set size     : (180, 76, 76)
Test set size      : (90, 76, 76)
Train set % of pwMS: 0.5444 (98)
Test set % of pwMS : 0.5444 (49)
Train loss at epoch 1: 0.6869
Train loss at epoch 2: 0.6804
Train loss at epoch 3: 0.6772
Train loss at epoch 4: 0.6736
Train loss at epoch 5: 0.6707
Train loss at epoch 6: 0.6686
Train loss at epoch 7: 0.6666
Train loss at epoch 8: 0.6651
Train loss at epoch 9: 0.6631
Train loss at epoch 10: 0.6614
Train loss at epoch 11: 0.6595
Train loss at epoch 12: 0.6577
Train loss at epoch 13: 0.6558
Train loss at epoch 14: 0.6536
Train loss at epoch 15: 0.6516
Train loss at epoch 16: 0.6492
Train loss at epoch 17: 0.6468
Train loss at epoch 18: 0.6448
Train loss at epoch 19: 0.6415
Train loss at epoch 20: 0.6388
Test AUC: 0.76
Fold: 2
Train set size     : (180, 76, 76)
Test set size      : (90, 76, 76)
Train set % of pwMS: 0.5444 (98)
Test set % of pwMS : 0.5444 (49)
Train loss at epoch 1: 0.6907
Train loss at epoch 2: 0.6882
Train loss at epoch 3: 0.687

In [19]:
import torch
import pandas as pd
import numpy as np
from torch_geometric.data import Data

# CAL DEFINIR ELS EMBEDDINGS ES FARAN SERVIR
# EN AQUEST CODI, NOMÉS ÉS UN VECTOR DE (node_dim X 1's)
node_dim = 1  # Asegúrate de que esto coincida con las dimensiones del archivo

def array_to_graph(data, y, th=0.0):
    num_nodes = data.shape[0]
    
    edge_index = []
    edge_weight = []
    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                if data[i, j] >= th:
                    edge_index.append([i, j])
                    edge_weight.append(data[i, j])
        
    y = torch.tensor([int(y)], dtype=torch.long)
    edge_index = torch.tensor(edge_index, dtype=torch.long)
    
    # Cargar datos de embeddings desde el archivo CSV
    embeddings_df = pd.read_csv('csvs/average_clustering_rs.csv', header=None) #modificar csv a necessitat
    x = torch.tensor(embeddings_df.values, dtype=torch.float)

    edge_weight = torch.tensor(edge_weight, dtype=torch.float)
    
    data = Data(x=x, edge_index=edge_index.t().contiguous(), edge_weight=edge_weight, y=y)
    #print(x)
    return data

In [20]:
data = data_RS_combined
print(data.shape)

target = demographics_N['mstype'].to_numpy()
print(target.shape)

(270, 76, 76)
(270,)


In [21]:
from torch_geometric.loader import DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve

skf = StratifiedKFold(n_splits=3)

NUM_EPOCHS = 20
preds = np.zeros(data.shape[0])
fold = 0

for train_index, test_index in skf.split(data, target):
    fold += 1
    print("Fold: {}".format(fold))

    # split dataset
    X_train, X_test = data[train_index], data[test_index]
    y_train, y_test = target[train_index], target[test_index]
    
    prop_train = np.where(y_train == 1)[0].shape[0] / y_train.shape[0]
    prop_test = np.where(y_test == 1)[0].shape[0] / y_test.shape[0]
    print("Train set size     : {}".format(X_train.shape))
    print("Test set size      : {}".format(X_test.shape))
    print("Train set % of pwMS: {:.4f} ({})".format(prop_train, y_train.sum()))
    print("Test set % of pwMS : {:.4f} ({})".format(prop_test, y_test.sum()))

    # list of Data structures (one for each subject)
    train_graphs = []
    for i in range(X_train.shape[0]):
        g = array_to_graph(X_train[i], y_train[i])
        train_graphs.append(g)
        
    test_graphs = []
    for i in range(X_test.shape[0]):
        g = array_to_graph(X_test[i], y_test[i])
        test_graphs.append(g)

    # create the model
    model = GCN()
    model = model.to(device)
    # optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    # loss function
    loss_fn = torch.nn.CrossEntropyLoss()

    # train function
    def train():
        model.train()
        
        train_loader = DataLoader(train_graphs, batch_size=32, shuffle=True)

        loss_all = 0
        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()
            output = model(batch)
            label = batch.y
            label = F.one_hot(label, num_classes=2)
            label = label.type(torch.FloatTensor)
            label = label.to(device)
            loss = loss_fn(output, label)
            loss.backward()
            loss_all += batch.num_graphs * loss.item()
            optimizer.step()

        return loss_all / len(train_graphs)

    # train for N epochs
    for epoch in range(NUM_EPOCHS):
        loss_value = train()
        print("Train loss at epoch {}: {:.4f}".format(epoch + 1, loss_value))

    # test phase 
    test_loader = DataLoader(test_graphs, batch_size=len(test_graphs), shuffle=False)
    
    for batch in test_loader:
        batch = batch.to(device)
        test_preds = F.softmax(model(batch), dim=1).detach().numpy()
    
    test_preds = test_preds[:, 1]
    preds[test_index] = test_preds
    
    auc_roc = roc_auc_score(y_test, test_preds)
    print("Test AUC: {:.2f}".format(auc_roc))

Fold: 1
Train set size     : (180, 76, 76)
Test set size      : (90, 76, 76)
Train set % of pwMS: 0.5444 (98)
Test set % of pwMS : 0.5444 (49)
Train loss at epoch 1: 0.7013
Train loss at epoch 2: 0.6966
Train loss at epoch 3: 0.6940
Train loss at epoch 4: 0.6927
Train loss at epoch 5: 0.6905
Train loss at epoch 6: 0.6898
Train loss at epoch 7: 0.6897
Train loss at epoch 8: 0.6893
Train loss at epoch 9: 0.6893
Train loss at epoch 10: 0.6892
Train loss at epoch 11: 0.6895
Train loss at epoch 12: 0.6892
Train loss at epoch 13: 0.6893
Train loss at epoch 14: 0.6894
Train loss at epoch 15: 0.6894
Train loss at epoch 16: 0.6893
Train loss at epoch 17: 0.6894
Train loss at epoch 18: 0.6893
Train loss at epoch 19: 0.6893
Train loss at epoch 20: 0.6893
Test AUC: 0.50
Fold: 2
Train set size     : (180, 76, 76)
Test set size      : (90, 76, 76)
Train set % of pwMS: 0.5444 (98)
Test set % of pwMS : 0.5444 (49)
Train loss at epoch 1: 0.6901
Train loss at epoch 2: 0.6896
Train loss at epoch 3: 0.689

## WORD2VEC

In [22]:
import torch
import pandas as pd
import numpy as np
from torch_geometric.data import Data

# CAL DEFINIR ELS EMBEDDINGS ES FARAN SERVIR
# EN AQUEST CODI, NOMÉS ÉS UN VECTOR DE (node_dim X 1's)
node_dim = 50  # Asegúrate de que esto coincida con las dimensiones del archivo

def array_to_graph(data, y, th=0.0):
    num_nodes = data.shape[0]
    
    edge_index = []
    edge_weight = []
    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                if data[i, j] >= th:
                    edge_index.append([i, j])
                    edge_weight.append(data[i, j])
        
    y = torch.tensor([int(y)], dtype=torch.long)
    edge_index = torch.tensor(edge_index, dtype=torch.long)
    
    # Cargar datos de embeddings desde el archivo CSV
    embeddings_df = pd.read_csv('csvs/embeddings_FA.csv', header=None) #modificar csv a necessitat
    x = torch.tensor(embeddings_df.values, dtype=torch.float)

    edge_weight = torch.tensor(edge_weight, dtype=torch.float)
    
    data = Data(x=x, edge_index=edge_index.t().contiguous(), edge_weight=edge_weight, y=y)
    #print(x)
    return data

In [23]:
data = data_FA_combined
print(data.shape)

target = demographics_N['mstype'].to_numpy()
print(target.shape)

(270, 76, 76)
(270,)


In [24]:
from torch_geometric.loader import DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve

skf = StratifiedKFold(n_splits=3)

NUM_EPOCHS = 20
preds = np.zeros(data.shape[0])
fold = 0

for train_index, test_index in skf.split(data, target):
    fold += 1
    print("Fold: {}".format(fold))

    # split dataset
    X_train, X_test = data[train_index], data[test_index]
    y_train, y_test = target[train_index], target[test_index]
    
    prop_train = np.where(y_train == 1)[0].shape[0] / y_train.shape[0]
    prop_test = np.where(y_test == 1)[0].shape[0] / y_test.shape[0]
    print("Train set size     : {}".format(X_train.shape))
    print("Test set size      : {}".format(X_test.shape))
    print("Train set % of pwMS: {:.4f} ({})".format(prop_train, y_train.sum()))
    print("Test set % of pwMS : {:.4f} ({})".format(prop_test, y_test.sum()))

    # list of Data structures (one for each subject)
    train_graphs = []
    for i in range(X_train.shape[0]):
        g = array_to_graph(X_train[i], y_train[i])
        train_graphs.append(g)
        
    test_graphs = []
    for i in range(X_test.shape[0]):
        g = array_to_graph(X_test[i], y_test[i])
        test_graphs.append(g)

    # create the model
    model = GCN()
    model = model.to(device)
    # optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    # loss function
    loss_fn = torch.nn.CrossEntropyLoss()

    # train function
    def train():
        model.train()
        
        train_loader = DataLoader(train_graphs, batch_size=32, shuffle=True)

        loss_all = 0
        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()
            output = model(batch)
            label = batch.y
            label = F.one_hot(label, num_classes=2)
            label = label.type(torch.FloatTensor)
            label = label.to(device)
            loss = loss_fn(output, label)
            loss.backward()
            loss_all += batch.num_graphs * loss.item()
            optimizer.step()

        return loss_all / len(train_graphs)

    # train for N epochs
    for epoch in range(NUM_EPOCHS):
        loss_value = train()
        print("Train loss at epoch {}: {:.4f}".format(epoch + 1, loss_value))

    # test phase 
    test_loader = DataLoader(test_graphs, batch_size=len(test_graphs), shuffle=False)
    
    for batch in test_loader:
        batch = batch.to(device)
        test_preds = F.softmax(model(batch), dim=1).detach().numpy()
    
    test_preds = test_preds[:, 1]
    preds[test_index] = test_preds
    
    auc_roc = roc_auc_score(y_test, test_preds)
    print("Test AUC: {:.2f}".format(auc_roc))

Fold: 1
Train set size     : (180, 76, 76)
Test set size      : (90, 76, 76)
Train set % of pwMS: 0.5444 (98)
Test set % of pwMS : 0.5444 (49)
Train loss at epoch 1: 1.1090
Train loss at epoch 2: 0.7309
Train loss at epoch 3: 0.6965
Train loss at epoch 4: 0.7150
Train loss at epoch 5: 0.6997
Train loss at epoch 6: 0.6808
Train loss at epoch 7: 0.6768
Train loss at epoch 8: 0.6726
Train loss at epoch 9: 0.6686
Train loss at epoch 10: 0.6642
Train loss at epoch 11: 0.6608
Train loss at epoch 12: 0.6579
Train loss at epoch 13: 0.6531
Train loss at epoch 14: 0.6508
Train loss at epoch 15: 0.6451
Train loss at epoch 16: 0.6417
Train loss at epoch 17: 0.6378
Train loss at epoch 18: 0.6324
Train loss at epoch 19: 0.6286
Train loss at epoch 20: 0.6242
Test AUC: 0.80
Fold: 2
Train set size     : (180, 76, 76)
Test set size      : (90, 76, 76)
Train set % of pwMS: 0.5444 (98)
Test set % of pwMS : 0.5444 (49)
Train loss at epoch 1: 0.7135
Train loss at epoch 2: 0.6962
Train loss at epoch 3: 0.680

In [25]:
import torch
import pandas as pd
import numpy as np
from torch_geometric.data import Data

# CAL DEFINIR ELS EMBEDDINGS ES FARAN SERVIR
# EN AQUEST CODI, NOMÉS ÉS UN VECTOR DE (node_dim X 1's)
node_dim = 50  # Asegúrate de que esto coincida con las dimensiones del archivo

def array_to_graph(data, y, th=0.0):
    num_nodes = data.shape[0]
    
    edge_index = []
    edge_weight = []
    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                if data[i, j] >= th:
                    edge_index.append([i, j])
                    edge_weight.append(data[i, j])
        
    y = torch.tensor([int(y)], dtype=torch.long)
    edge_index = torch.tensor(edge_index, dtype=torch.long)
    
    # Cargar datos de embeddings desde el archivo CSV
    embeddings_df = pd.read_csv('csvs/embeddings_GM.csv', header=None) #modificar csv a necessitat
    x = torch.tensor(embeddings_df.values, dtype=torch.float)

    edge_weight = torch.tensor(edge_weight, dtype=torch.float)
    
    data = Data(x=x, edge_index=edge_index.t().contiguous(), edge_weight=edge_weight, y=y)
    #print(x)
    return data

In [26]:
data = data_GM_combined
print(data.shape)

target = demographics_N['mstype'].to_numpy()
print(target.shape)

(270, 76, 76)
(270,)


In [27]:
from torch_geometric.loader import DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve

skf = StratifiedKFold(n_splits=3)

NUM_EPOCHS = 20
preds = np.zeros(data.shape[0])
fold = 0

for train_index, test_index in skf.split(data, target):
    fold += 1
    print("Fold: {}".format(fold))

    # split dataset
    X_train, X_test = data[train_index], data[test_index]
    y_train, y_test = target[train_index], target[test_index]
    
    prop_train = np.where(y_train == 1)[0].shape[0] / y_train.shape[0]
    prop_test = np.where(y_test == 1)[0].shape[0] / y_test.shape[0]
    print("Train set size     : {}".format(X_train.shape))
    print("Test set size      : {}".format(X_test.shape))
    print("Train set % of pwMS: {:.4f} ({})".format(prop_train, y_train.sum()))
    print("Test set % of pwMS : {:.4f} ({})".format(prop_test, y_test.sum()))

    # list of Data structures (one for each subject)
    train_graphs = []
    for i in range(X_train.shape[0]):
        g = array_to_graph(X_train[i], y_train[i])
        train_graphs.append(g)
        
    test_graphs = []
    for i in range(X_test.shape[0]):
        g = array_to_graph(X_test[i], y_test[i])
        test_graphs.append(g)

    # create the model
    model = GCN()
    model = model.to(device)
    # optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    # loss function
    loss_fn = torch.nn.CrossEntropyLoss()

    # train function
    def train():
        model.train()
        
        train_loader = DataLoader(train_graphs, batch_size=32, shuffle=True)

        loss_all = 0
        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()
            output = model(batch)
            label = batch.y
            label = F.one_hot(label, num_classes=2)
            label = label.type(torch.FloatTensor)
            label = label.to(device)
            loss = loss_fn(output, label)
            loss.backward()
            loss_all += batch.num_graphs * loss.item()
            optimizer.step()

        return loss_all / len(train_graphs)

    # train for N epochs
    for epoch in range(NUM_EPOCHS):
        loss_value = train()
        print("Train loss at epoch {}: {:.4f}".format(epoch + 1, loss_value))

    # test phase 
    test_loader = DataLoader(test_graphs, batch_size=len(test_graphs), shuffle=False)
    
    for batch in test_loader:
        batch = batch.to(device)
        test_preds = F.softmax(model(batch), dim=1).detach().numpy()
    
    test_preds = test_preds[:, 1]
    preds[test_index] = test_preds
    
    auc_roc = roc_auc_score(y_test, test_preds)
    print("Test AUC: {:.2f}".format(auc_roc))

Fold: 1
Train set size     : (180, 76, 76)
Test set size      : (90, 76, 76)
Train set % of pwMS: 0.5444 (98)
Test set % of pwMS : 0.5444 (49)
Train loss at epoch 1: 1.0291
Train loss at epoch 2: 0.5033
Train loss at epoch 3: 0.5709
Train loss at epoch 4: 0.5403
Train loss at epoch 5: 0.4620
Train loss at epoch 6: 0.3844
Train loss at epoch 7: 0.3457
Train loss at epoch 8: 0.3288
Train loss at epoch 9: 0.3000
Train loss at epoch 10: 0.2749
Train loss at epoch 11: 0.2572
Train loss at epoch 12: 0.2387
Train loss at epoch 13: 0.2221
Train loss at epoch 14: 0.2055
Train loss at epoch 15: 0.1959
Train loss at epoch 16: 0.1808
Train loss at epoch 17: 0.1713
Train loss at epoch 18: 0.1695
Train loss at epoch 19: 0.1584
Train loss at epoch 20: 0.1545
Test AUC: 0.76
Fold: 2
Train set size     : (180, 76, 76)
Test set size      : (90, 76, 76)
Train set % of pwMS: 0.5444 (98)
Test set % of pwMS : 0.5444 (49)
Train loss at epoch 1: 0.5933
Train loss at epoch 2: 0.5927
Train loss at epoch 3: 0.558

In [28]:
import torch
import pandas as pd
import numpy as np
from torch_geometric.data import Data

# CAL DEFINIR ELS EMBEDDINGS ES FARAN SERVIR
# EN AQUEST CODI, NOMÉS ÉS UN VECTOR DE (node_dim X 1's)
node_dim = 50  # Asegúrate de que esto coincida con las dimensiones del archivo

def array_to_graph(data, y, th=0.0):
    num_nodes = data.shape[0]
    
    edge_index = []
    edge_weight = []
    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                if data[i, j] >= th:
                    edge_index.append([i, j])
                    edge_weight.append(data[i, j])
        
    y = torch.tensor([int(y)], dtype=torch.long)
    edge_index = torch.tensor(edge_index, dtype=torch.long)
    
    # Cargar datos de embeddings desde el archivo CSV
    embeddings_df = pd.read_csv('csvs/embeddings_RS.csv', header=None) #modificar csv a necessitat
    x = torch.tensor(embeddings_df.values, dtype=torch.float)

    edge_weight = torch.tensor(edge_weight, dtype=torch.float)
    
    data = Data(x=x, edge_index=edge_index.t().contiguous(), edge_weight=edge_weight, y=y)
    #print(x)
    return data

In [29]:
data = data_RS_combined
print(data.shape)

target = demographics_N['mstype'].to_numpy()
print(target.shape)

(270, 76, 76)
(270,)


In [30]:
from torch_geometric.loader import DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve

skf = StratifiedKFold(n_splits=3)

NUM_EPOCHS = 20
preds = np.zeros(data.shape[0])
fold = 0

for train_index, test_index in skf.split(data, target):
    fold += 1
    print("Fold: {}".format(fold))

    # split dataset
    X_train, X_test = data[train_index], data[test_index]
    y_train, y_test = target[train_index], target[test_index]
    
    prop_train = np.where(y_train == 1)[0].shape[0] / y_train.shape[0]
    prop_test = np.where(y_test == 1)[0].shape[0] / y_test.shape[0]
    print("Train set size     : {}".format(X_train.shape))
    print("Test set size      : {}".format(X_test.shape))
    print("Train set % of pwMS: {:.4f} ({})".format(prop_train, y_train.sum()))
    print("Test set % of pwMS : {:.4f} ({})".format(prop_test, y_test.sum()))

    # list of Data structures (one for each subject)
    train_graphs = []
    for i in range(X_train.shape[0]):
        g = array_to_graph(X_train[i], y_train[i])
        train_graphs.append(g)
        
    test_graphs = []
    for i in range(X_test.shape[0]):
        g = array_to_graph(X_test[i], y_test[i])
        test_graphs.append(g)

    # create the model
    model = GCN()
    model = model.to(device)
    # optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    # loss function
    loss_fn = torch.nn.CrossEntropyLoss()

    # train function
    def train():
        model.train()
        
        train_loader = DataLoader(train_graphs, batch_size=32, shuffle=True)

        loss_all = 0
        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()
            output = model(batch)
            label = batch.y
            label = F.one_hot(label, num_classes=2)
            label = label.type(torch.FloatTensor)
            label = label.to(device)
            loss = loss_fn(output, label)
            loss.backward()
            loss_all += batch.num_graphs * loss.item()
            optimizer.step()

        return loss_all / len(train_graphs)

    # train for N epochs
    for epoch in range(NUM_EPOCHS):
        loss_value = train()
        print("Train loss at epoch {}: {:.4f}".format(epoch + 1, loss_value))

    # test phase 
    test_loader = DataLoader(test_graphs, batch_size=len(test_graphs), shuffle=False)
    
    for batch in test_loader:
        batch = batch.to(device)
        test_preds = F.softmax(model(batch), dim=1).detach().numpy()
    
    test_preds = test_preds[:, 1]
    preds[test_index] = test_preds
    
    auc_roc = roc_auc_score(y_test, test_preds)
    print("Test AUC: {:.2f}".format(auc_roc))

Fold: 1
Train set size     : (180, 76, 76)
Test set size      : (90, 76, 76)
Train set % of pwMS: 0.5444 (98)
Test set % of pwMS : 0.5444 (49)
Train loss at epoch 1: 0.6847
Train loss at epoch 2: 0.6573
Train loss at epoch 3: 0.6421
Train loss at epoch 4: 0.6318
Train loss at epoch 5: 0.6207
Train loss at epoch 6: 0.6174
Train loss at epoch 7: 0.6070
Train loss at epoch 8: 0.5893
Train loss at epoch 9: 0.5909
Train loss at epoch 10: 0.5771
Train loss at epoch 11: 0.5722
Train loss at epoch 12: 0.5623
Train loss at epoch 13: 0.5598
Train loss at epoch 14: 0.5566
Train loss at epoch 15: 0.5478
Train loss at epoch 16: 0.5481
Train loss at epoch 17: 0.5338
Train loss at epoch 18: 0.5332
Train loss at epoch 19: 0.5249
Train loss at epoch 20: 0.5270
Test AUC: 0.63
Fold: 2
Train set size     : (180, 76, 76)
Test set size      : (90, 76, 76)
Train set % of pwMS: 0.5444 (98)
Test set % of pwMS : 0.5444 (49)
Train loss at epoch 1: 0.6934
Train loss at epoch 2: 0.6810
Train loss at epoch 3: 0.670

## NODE2VEC

In [31]:
import torch
import pandas as pd
import numpy as np
from torch_geometric.data import Data

# CAL DEFINIR ELS EMBEDDINGS ES FARAN SERVIR
# EN AQUEST CODI, NOMÉS ÉS UN VECTOR DE (node_dim X 1's)
node_dim = 25  # Asegúrate de que esto coincida con las dimensiones del archivo

def array_to_graph(data, y, th=0.0):
    num_nodes = data.shape[0]
    
    edge_index = []
    edge_weight = []
    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                if data[i, j] >= th:
                    edge_index.append([i, j])
                    edge_weight.append(data[i, j])
        
    y = torch.tensor([int(y)], dtype=torch.long)
    edge_index = torch.tensor(edge_index, dtype=torch.long)
    
    # Cargar datos de embeddings desde el archivo CSV
    embeddings_df = pd.read_csv('csvs/embeddings_FA_n2v.csv', header=None) #modificar csv a necessitat
    x = torch.tensor(embeddings_df.values, dtype=torch.float)

    edge_weight = torch.tensor(edge_weight, dtype=torch.float)
    
    data = Data(x=x, edge_index=edge_index.t().contiguous(), edge_weight=edge_weight, y=y)
    #print(x)
    return data

In [32]:
data = data_FA_combined
print(data.shape)

target = demographics_N['mstype'].to_numpy()
print(target.shape)

(270, 76, 76)
(270,)


In [33]:
from torch_geometric.loader import DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve

skf = StratifiedKFold(n_splits=3)

NUM_EPOCHS = 20
preds = np.zeros(data.shape[0])
fold = 0

for train_index, test_index in skf.split(data, target):
    fold += 1
    print("Fold: {}".format(fold))

    # split dataset
    X_train, X_test = data[train_index], data[test_index]
    y_train, y_test = target[train_index], target[test_index]
    
    prop_train = np.where(y_train == 1)[0].shape[0] / y_train.shape[0]
    prop_test = np.where(y_test == 1)[0].shape[0] / y_test.shape[0]
    print("Train set size     : {}".format(X_train.shape))
    print("Test set size      : {}".format(X_test.shape))
    print("Train set % of pwMS: {:.4f} ({})".format(prop_train, y_train.sum()))
    print("Test set % of pwMS : {:.4f} ({})".format(prop_test, y_test.sum()))

    # list of Data structures (one for each subject)
    train_graphs = []
    for i in range(X_train.shape[0]):
        g = array_to_graph(X_train[i], y_train[i])
        train_graphs.append(g)
        
    test_graphs = []
    for i in range(X_test.shape[0]):
        g = array_to_graph(X_test[i], y_test[i])
        test_graphs.append(g)

    # create the model
    model = GCN()
    model = model.to(device)
    # optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    # loss function
    loss_fn = torch.nn.CrossEntropyLoss()

    # train function
    def train():
        model.train()
        
        train_loader = DataLoader(train_graphs, batch_size=32, shuffle=True)

        loss_all = 0
        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()
            output = model(batch)
            label = batch.y
            label = F.one_hot(label, num_classes=2)
            label = label.type(torch.FloatTensor)
            label = label.to(device)
            loss = loss_fn(output, label)
            loss.backward()
            loss_all += batch.num_graphs * loss.item()
            optimizer.step()

        return loss_all / len(train_graphs)

    # train for N epochs
    for epoch in range(NUM_EPOCHS):
        loss_value = train()
        print("Train loss at epoch {}: {:.4f}".format(epoch + 1, loss_value))

    # test phase 
    test_loader = DataLoader(test_graphs, batch_size=len(test_graphs), shuffle=False)
    
    for batch in test_loader:
        batch = batch.to(device)
        test_preds = F.softmax(model(batch), dim=1).detach().numpy()
    
    test_preds = test_preds[:, 1]
    preds[test_index] = test_preds
    
    auc_roc = roc_auc_score(y_test, test_preds)
    print("Test AUC: {:.2f}".format(auc_roc))

Fold: 1
Train set size     : (180, 76, 76)
Test set size      : (90, 76, 76)
Train set % of pwMS: 0.5444 (98)
Test set % of pwMS : 0.5444 (49)
Train loss at epoch 1: 0.6784
Train loss at epoch 2: 0.6698
Train loss at epoch 3: 0.6668
Train loss at epoch 4: 0.6593
Train loss at epoch 5: 0.6532
Train loss at epoch 6: 0.6501
Train loss at epoch 7: 0.6434
Train loss at epoch 8: 0.6380
Train loss at epoch 9: 0.6315
Train loss at epoch 10: 0.6258
Train loss at epoch 11: 0.6213
Train loss at epoch 12: 0.6138
Train loss at epoch 13: 0.6068
Train loss at epoch 14: 0.6002
Train loss at epoch 15: 0.5948
Train loss at epoch 16: 0.5858
Train loss at epoch 17: 0.5799
Train loss at epoch 18: 0.5721
Train loss at epoch 19: 0.5638
Train loss at epoch 20: 0.5584
Test AUC: 0.79
Fold: 2
Train set size     : (180, 76, 76)
Test set size      : (90, 76, 76)
Train set % of pwMS: 0.5444 (98)
Test set % of pwMS : 0.5444 (49)
Train loss at epoch 1: 0.6924
Train loss at epoch 2: 0.6791
Train loss at epoch 3: 0.680

In [34]:
import torch
import pandas as pd
import numpy as np
from torch_geometric.data import Data

# CAL DEFINIR ELS EMBEDDINGS ES FARAN SERVIR
# EN AQUEST CODI, NOMÉS ÉS UN VECTOR DE (node_dim X 1's)
node_dim = 25  # Asegúrate de que esto coincida con las dimensiones del archivo

def array_to_graph(data, y, th=0.0):
    num_nodes = data.shape[0]
    
    edge_index = []
    edge_weight = []
    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                if data[i, j] >= th:
                    edge_index.append([i, j])
                    edge_weight.append(data[i, j])
        
    y = torch.tensor([int(y)], dtype=torch.long)
    edge_index = torch.tensor(edge_index, dtype=torch.long)
    
    # Cargar datos de embeddings desde el archivo CSV
    embeddings_df = pd.read_csv('csvs/embeddings_GM_n2v.csv', header=None) #modificar csv a necessitat
    x = torch.tensor(embeddings_df.values, dtype=torch.float)

    edge_weight = torch.tensor(edge_weight, dtype=torch.float)
    
    data = Data(x=x, edge_index=edge_index.t().contiguous(), edge_weight=edge_weight, y=y)
    #print(x)
    return data

In [35]:
data = data_GM_combined
print(data.shape)

target = demographics_N['mstype'].to_numpy()
print(target.shape)

(270, 76, 76)
(270,)


In [36]:
from torch_geometric.loader import DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve

skf = StratifiedKFold(n_splits=3)

NUM_EPOCHS = 20
preds = np.zeros(data.shape[0])
fold = 0

for train_index, test_index in skf.split(data, target):
    fold += 1
    print("Fold: {}".format(fold))

    # split dataset
    X_train, X_test = data[train_index], data[test_index]
    y_train, y_test = target[train_index], target[test_index]
    
    prop_train = np.where(y_train == 1)[0].shape[0] / y_train.shape[0]
    prop_test = np.where(y_test == 1)[0].shape[0] / y_test.shape[0]
    print("Train set size     : {}".format(X_train.shape))
    print("Test set size      : {}".format(X_test.shape))
    print("Train set % of pwMS: {:.4f} ({})".format(prop_train, y_train.sum()))
    print("Test set % of pwMS : {:.4f} ({})".format(prop_test, y_test.sum()))

    # list of Data structures (one for each subject)
    train_graphs = []
    for i in range(X_train.shape[0]):
        g = array_to_graph(X_train[i], y_train[i])
        train_graphs.append(g)
        
    test_graphs = []
    for i in range(X_test.shape[0]):
        g = array_to_graph(X_test[i], y_test[i])
        test_graphs.append(g)

    # create the model
    model = GCN()
    model = model.to(device)
    # optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    # loss function
    loss_fn = torch.nn.CrossEntropyLoss()

    # train function
    def train():
        model.train()
        
        train_loader = DataLoader(train_graphs, batch_size=32, shuffle=True)

        loss_all = 0
        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()
            output = model(batch)
            label = batch.y
            label = F.one_hot(label, num_classes=2)
            label = label.type(torch.FloatTensor)
            label = label.to(device)
            loss = loss_fn(output, label)
            loss.backward()
            loss_all += batch.num_graphs * loss.item()
            optimizer.step()

        return loss_all / len(train_graphs)

    # train for N epochs
    for epoch in range(NUM_EPOCHS):
        loss_value = train()
        print("Train loss at epoch {}: {:.4f}".format(epoch + 1, loss_value))

    # test phase 
    test_loader = DataLoader(test_graphs, batch_size=len(test_graphs), shuffle=False)
    
    for batch in test_loader:
        batch = batch.to(device)
        test_preds = F.softmax(model(batch), dim=1).detach().numpy()
    
    test_preds = test_preds[:, 1]
    preds[test_index] = test_preds
    
    auc_roc = roc_auc_score(y_test, test_preds)
    print("Test AUC: {:.2f}".format(auc_roc))

Fold: 1
Train set size     : (180, 76, 76)
Test set size      : (90, 76, 76)
Train set % of pwMS: 0.5444 (98)
Test set % of pwMS : 0.5444 (49)
Train loss at epoch 1: 0.4104
Train loss at epoch 2: 0.3723
Train loss at epoch 3: 0.3483
Train loss at epoch 4: 0.3268
Train loss at epoch 5: 0.3027
Train loss at epoch 6: 0.2781
Train loss at epoch 7: 0.2611
Train loss at epoch 8: 0.2370
Train loss at epoch 9: 0.2191
Train loss at epoch 10: 0.2066
Train loss at epoch 11: 0.1918
Train loss at epoch 12: 0.1807
Train loss at epoch 13: 0.1725
Train loss at epoch 14: 0.1630
Train loss at epoch 15: 0.1560
Train loss at epoch 16: 0.1507
Train loss at epoch 17: 0.1502
Train loss at epoch 18: 0.1456
Train loss at epoch 19: 0.1419
Train loss at epoch 20: 0.1386
Test AUC: 0.76
Fold: 2
Train set size     : (180, 76, 76)
Test set size      : (90, 76, 76)
Train set % of pwMS: 0.5444 (98)
Test set % of pwMS : 0.5444 (49)
Train loss at epoch 1: 1.0161
Train loss at epoch 2: 0.6791
Train loss at epoch 3: 0.691

In [37]:
import torch
import pandas as pd
import numpy as np
from torch_geometric.data import Data

# CAL DEFINIR ELS EMBEDDINGS ES FARAN SERVIR
# EN AQUEST CODI, NOMÉS ÉS UN VECTOR DE (node_dim X 1's)
node_dim = 25  # Asegúrate de que esto coincida con las dimensiones del archivo

def array_to_graph(data, y, th=0.0):
    num_nodes = data.shape[0]
    
    edge_index = []
    edge_weight = []
    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                if data[i, j] >= th:
                    edge_index.append([i, j])
                    edge_weight.append(data[i, j])
        
    y = torch.tensor([int(y)], dtype=torch.long)
    edge_index = torch.tensor(edge_index, dtype=torch.long)
    
    # Cargar datos de embeddings desde el archivo CSV
    embeddings_df = pd.read_csv('csvs/embeddings_RS_n2v.csv', header=None) #modificar csv a necessitat
    x = torch.tensor(embeddings_df.values, dtype=torch.float)

    edge_weight = torch.tensor(edge_weight, dtype=torch.float)
    
    data = Data(x=x, edge_index=edge_index.t().contiguous(), edge_weight=edge_weight, y=y)
    #print(x)
    return data

In [38]:
data = data_RS_combined
print(data.shape)

target = demographics_N['mstype'].to_numpy()
print(target.shape)

(270, 76, 76)
(270,)


In [39]:
from torch_geometric.loader import DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve

skf = StratifiedKFold(n_splits=3)

NUM_EPOCHS = 20
preds = np.zeros(data.shape[0])
fold = 0

for train_index, test_index in skf.split(data, target):
    fold += 1
    print("Fold: {}".format(fold))

    # split dataset
    X_train, X_test = data[train_index], data[test_index]
    y_train, y_test = target[train_index], target[test_index]
    
    prop_train = np.where(y_train == 1)[0].shape[0] / y_train.shape[0]
    prop_test = np.where(y_test == 1)[0].shape[0] / y_test.shape[0]
    print("Train set size     : {}".format(X_train.shape))
    print("Test set size      : {}".format(X_test.shape))
    print("Train set % of pwMS: {:.4f} ({})".format(prop_train, y_train.sum()))
    print("Test set % of pwMS : {:.4f} ({})".format(prop_test, y_test.sum()))

    # list of Data structures (one for each subject)
    train_graphs = []
    for i in range(X_train.shape[0]):
        g = array_to_graph(X_train[i], y_train[i])
        train_graphs.append(g)
        
    test_graphs = []
    for i in range(X_test.shape[0]):
        g = array_to_graph(X_test[i], y_test[i])
        test_graphs.append(g)

    # create the model
    model = GCN()
    model = model.to(device)
    # optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    # loss function
    loss_fn = torch.nn.CrossEntropyLoss()

    # train function
    def train():
        model.train()
        
        train_loader = DataLoader(train_graphs, batch_size=32, shuffle=True)

        loss_all = 0
        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()
            output = model(batch)
            label = batch.y
            label = F.one_hot(label, num_classes=2)
            label = label.type(torch.FloatTensor)
            label = label.to(device)
            loss = loss_fn(output, label)
            loss.backward()
            loss_all += batch.num_graphs * loss.item()
            optimizer.step()

        return loss_all / len(train_graphs)

    # train for N epochs
    for epoch in range(NUM_EPOCHS):
        loss_value = train()
        print("Train loss at epoch {}: {:.4f}".format(epoch + 1, loss_value))

    # test phase 
    test_loader = DataLoader(test_graphs, batch_size=len(test_graphs), shuffle=False)
    
    for batch in test_loader:
        batch = batch.to(device)
        test_preds = F.softmax(model(batch), dim=1).detach().numpy()
    
    test_preds = test_preds[:, 1]
    preds[test_index] = test_preds
    
    auc_roc = roc_auc_score(y_test, test_preds)
    print("Test AUC: {:.2f}".format(auc_roc))

Fold: 1
Train set size     : (180, 76, 76)
Test set size      : (90, 76, 76)
Train set % of pwMS: 0.5444 (98)
Test set % of pwMS : 0.5444 (49)
Train loss at epoch 1: 0.7023
Train loss at epoch 2: 0.6863
Train loss at epoch 3: 0.6755
Train loss at epoch 4: 0.6678
Train loss at epoch 5: 0.6590
Train loss at epoch 6: 0.6498
Train loss at epoch 7: 0.6393
Train loss at epoch 8: 0.6315
Train loss at epoch 9: 0.6237
Train loss at epoch 10: 0.6156
Train loss at epoch 11: 0.6084
Train loss at epoch 12: 0.5990
Train loss at epoch 13: 0.5921
Train loss at epoch 14: 0.5852
Train loss at epoch 15: 0.5777
Train loss at epoch 16: 0.5726
Train loss at epoch 17: 0.5649
Train loss at epoch 18: 0.5603
Train loss at epoch 19: 0.5519
Train loss at epoch 20: 0.5460
Test AUC: 0.63
Fold: 2
Train set size     : (180, 76, 76)
Test set size      : (90, 76, 76)
Train set % of pwMS: 0.5444 (98)
Test set % of pwMS : 0.5444 (49)
Train loss at epoch 1: 0.7256
Train loss at epoch 2: 0.7065
Train loss at epoch 3: 0.698